In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageFilter
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import os

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = os.listdir(root_dir)
        self.label_map = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        self.image_paths = []
        self.labels = []

        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for filename in os.listdir(class_dir):
                    if filename.endswith(".jpg") or filename.endswith(".png"):
                        self.image_paths.append(os.path.join(class_dir, filename))
                        self.labels.append(self.label_map[class_name])

    def __len__(self):
        return len(self.image_paths)

    def crop_to_retina(self, img):
        try:
            # Convert PIL image to OpenCV format
            img_cv = np.array(img)
            if img_cv.ndim != 3 or img_cv.shape[2] != 3:
                raise ValueError("Invalid image format")

            img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)

            # Detect circles using Hough Transform
            circles = cv2.HoughCircles(
                gray,
                cv2.HOUGH_GRADIENT,
                dp=1,
                minDist=gray.shape[0] // 4,
                param1=50,
                param2=30,
                minRadius=30,
                maxRadius=gray.shape[0] // 2
            )

            if circles is not None:
                circles = np.uint16(np.around(circles))
                x, y, r = map(int, circles[0][0])
                x1, y1 = max(int(x - r), 0), max(int(y - r), 0)
                x2, y2 = min(int(x + r), img_cv.shape[1]), min(int(y + r), img_cv.shape[0])
                cropped = img_cv[y1:y2, x1:x2]
            else:
                cropped = img_cv  # fallback if circle not detected

            if cropped.size == 0:
                raise ValueError("Cropped image is empty")

            cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
            return Image.fromarray(cropped)

        except Exception as e:
            print(f"[WARN] crop_to_retina failed: {e}")
            return img  # Return original image if error occurs


    def __getitem__(self, idx):
      img_path = self.image_paths[idx]
      label = self.labels[idx]
      try:
          image = Image.open(img_path).convert("RGB")
          image = image.filter(ImageFilter.GaussianBlur(radius=1.5))
          image = self.crop_to_retina(image)
          if self.transform:
              image = self.transform(image)
          return image, label
      except Exception as e:
          print(f"[ERROR] Failed to process {img_path}: {e}")
          raise e



from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = ImageDataset(root_dir='/content/drive/MyDrive/diabetic_retinopathy/gaussian_filtered_images/gaussian_filtered_images', transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

import torch

def get_numpy_from_loader(loader):
    all_images = []
    all_labels = []

    for batch in loader:
        imgs, labels = batch
        # Convert from [B, C, H, W] to [B, H, W, C]
        imgs_np = imgs.permute(0, 2, 3, 1).numpy()
        all_images.append(imgs_np)
        all_labels.extend(labels.numpy())

    return np.vstack(all_images), np.array(all_labels)

images_np, labels_np = get_numpy_from_loader(loader)

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Flatten

# Preprocess for VGG
images_preprocessed = preprocess_input(images_np * 255.0)

# VGG16 Model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
flattened_output = Flatten()(base_model.output)
feature_model = Model(inputs=base_model.input, outputs=flattened_output)

# Extract features
features = feature_model.predict(images_preprocessed, batch_size=32, verbose=1)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler

# Step 1: Use ExtraTreesClassifier to select top 4 features
selector_model = ExtraTreesClassifier(n_estimators=100)
selector_model.fit(features, labels_np)

# Get top 4 important feature indices
top_k = 4
indices = np.argsort(selector_model.feature_importances_)[-top_k:]
features_selected = features[:, indices]  # shape: (num_samples, 4)

# Step 2: Normalize to [0, π] for angle encoding
scaler = MinMaxScaler(feature_range=(0, np.pi))
features_scaled = scaler.fit_transform(features_selected)

# Step 3: Convert to torch.Tensor
X_tensor = torch.tensor(features_scaled, dtype=torch.float32)
y_tensor = torch.tensor(labels_np, dtype=torch.long)




58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
115/115 ━━━━━━━━━━━━━━━━━━━━ 2209s 19s/step


In [ ]:
!pip install qiskit==1.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install qiskit-machine-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator
num_classes=5
# Step 4: Train/test split and create DataLoaders
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=8, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=8)


# === Quantum Circuit ===
def conv_circuit(params):
    qc = QuantumCircuit(2)
    qc.rz(-np.pi / 2, 1)
    qc.cx(1, 0)
    qc.rz(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.ry(params[2], 1)
    qc.cx(1, 0)
    qc.rz(np.pi / 2, 0)
    return qc

def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits)
    params = ParameterVector(param_prefix, length=(num_qubits // 2) * 3)
    idx = 0
    for q1, q2 in zip(range(0, num_qubits, 2), range(1, num_qubits, 2)):
        qc.append(conv_circuit(params[idx:idx+3]), [q1, q2])
        idx += 3
    return qc

def pool_circuit(params):
    qc = QuantumCircuit(2)
    qc.rz(-np.pi / 2, 1)
    qc.cx(1, 0)
    qc.rz(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.ry(params[2], 1)
    return qc

def pool_layer(sources, sinks, param_prefix):
    num_pairs = len(sources)
    qc = QuantumCircuit(max(sources + sinks) + 1)
    params = ParameterVector(param_prefix, length=num_pairs * 3)
    for i, (src, snk) in enumerate(zip(sources, sinks)):
        qc.append(pool_circuit(params[i * 3:(i + 1) * 3]), [src, snk])
    return qc

# Create QNN circuit
feature_map = ZFeatureMap(4)
ansatz = QuantumCircuit(4)

# Use convolution and pooling only on qubits 0-3
ansatz.compose(conv_layer(4, "c1"), inplace=True)
ansatz.compose(pool_layer([0, 1], [2, 3], "p1"), inplace=True)
ansatz.compose(conv_layer(4, "c2"), inplace=True)
ansatz.compose(pool_layer([0, 2], [1, 3], "p2"), inplace=True)
ansatz.compose(conv_layer(4, "c3"), inplace=True)

# Final quantum circuit (4-qubit only)
qc = QuantumCircuit(4)
qc.compose(feature_map, range(4), inplace=True)
qc.compose(ansatz, range(4), inplace=True)
# Final quantum circuit
qc = QuantumCircuit(4)
qc.compose(feature_map, range(4), inplace=True)
qc.compose(ansatz, range(4), inplace=True)

# Define observables (Z measurement on each qubit)
observables = []
for i in range(num_classes):
    pauli = ["I"] * 4
    pauli[i % 4] = "Z"
    observables.append(SparsePauliOp.from_list([("".join(pauli), 1)]))

estimator = Estimator()
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    observables=observables,
    estimator=estimator
)

# === Hybrid Model ===
class ClassicalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(num_classes, 32)
        self.fc2 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

class HybridModel(nn.Module):
    def __init__(self, qnn, classical_nn):
        super().__init__()
        self.qnn = TorchConnector(qnn)
        self.classical = classical_nn

    def forward(self, x):
        qnn_out = self.qnn(x)  # No need to slice anymore
        return self.classical(qnn_out)

# Instantiate model, loss, and optimizer
hybrid_model = HybridModel(qnn, ClassicalNN())
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(hybrid_model.parameters(), lr=0.01)

# === Training Loop ===
epochs = 15
for epoch in range(epochs):
    hybrid_model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = hybrid_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(running_loss)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {acc:.4f}")


/tmp/ipython-input-4-2728913253.py:94: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/tmp/ipython-input-4-2728913253.py:95: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


1.4988934993743896
2.994351863861084
4.383085250854492
5.670616626739502
7.274426460266113
8.76193642616272
10.00337815284729
11.41196072101593
12.723196864128113
14.264400959014893
15.620671272277832
17.22242844104767
18.721619725227356
20.024062037467957
20.999786734580994
22.066046357154846
23.4588543176651
24.783827424049377
25.87415850162506
26.783437371253967
27.702481746673584
28.804332494735718
30.14372968673706
31.223801493644714
32.52124464511871
33.19126957654953
33.8283816576004
35.02104687690735
36.37402546405792
37.751293420791626
38.51602792739868
39.50312751531601
40.500597178936005
41.46055310964584
42.86296707391739
44.961287558078766
46.05896347761154
47.13406926393509
48.57773548364639
50.27451950311661
51.78525060415268
53.12310379743576
54.04415017366409
55.17826372385025
56.39434593915939
57.4523041844368
58.328008353710175
59.053048849105835
60.263591051101685
60.9122508764267
62.210134506225586
63.13146913051605
63.71061134338379
65.43137168884277
66.4237433075